## Work with Pyemu to make .pst object and stuff....

### Using MAP SWB observations and parameters

In [2]:
# First run-through 11/26/2019
# More parameters 1/29/2020
# More observations 2/3/2020
# Lots of updates July 2020:
#   - new lookup table
#   - adjustments to observations sorting and numbers

# 8/10/2020 Added lots more observations - SW watersheds baseflow and runoff.
# 8/20/20 Re-worked to create more generalized process, moved weights to separate notebook

# 9/9/2020 Ran with full set of MAP parameters (494 instead of 56)

In [1]:
# Reinstalled pyemu 7/15/2020 with pip
import pyemu as pyemu
import os
import pandas as pd
pyemu.__version__

'0+unknown'

## Using the "from_io" utility

In [2]:
# Set up paths:
PARDIR = 'E:\\UMID_Data\\0188_offline\\modeldev\\PARS'
OBSDIR = 'E:\\UMID_Data\\0188_offline\\modeldev\\OBS'
outdir = 'E:\\UMID_Data\\0188_offline\\modeldev\\PEST\\PEST_FILE_UPDATES'

## IMPORTANT TO UPDATE THESE FILE NAMES:

In [3]:
# Set current versions of lookup tables:
#landuse_version = 'YETI_Landuse_lookup_table_V4'
#irrigation_version = 'YETI_Irrigation_lookup_table_V2'
landuse_version = 'YETI_LU_lookup_table_MOREPARS'
irrigation_version = 'YETI_IRR_lookup_table_MOREPARS'

lufile = landuse_version + '.xlsx'
irrfile = irrigation_version + '.xlsx'

### Run notebook #1 again if parameters/values have changed

In [4]:
# Lookup tables saved from jupyter notebook 1.Export lookup tables and tpl files from excel.ipynb:

# LAND USE infiles
infile = os.path.join(outdir, (landuse_version + '.txt')) 
tplfile = os.path.join(outdir,(landuse_version + '.tpl'))

# Irrigation ET in-file NEW_Irrigation_lookup_table_PestTest.txt:
infile2 = os.path.join(outdir,(irrigation_version + '.txt'))
tplfile2 = os.path.join(outdir,(irrigation_version + '.tpl'))

# Observations - TWO types now:
# in the P:\0188\modeldev\OBS directory:
# "ins_file" = MAP_simulated_et_obs.ins
insfile1 = os.path.join(OBSDIR,'MAP_simulated_et_obs_ALL_ET_sorted.ins')
insfile2 = os.path.join(OBSDIR,'MAP_simulated_obs_SW.ins')

# "out_file" = swbstats_simulated_obs_et.obs
obsfile1 = os.path.join(OBSDIR,'swbstats_simulated_obs_et_ALL_sorted.obs')
obsfile2 = os.path.join(OBSDIR,'swbstats_simulated_obs_SW.obs')

# OTHER data:
# obsdir --> \ALLOBS.INFO.for.pyemu.csv  --> OBSVAL, OBSNME, WEIGHT, has lots of
# entries that are "zero" values and will be skipped

# pardir -->\MAP_parlist.csv --> PARNME, PARVAL1, PARGPNME, PARLBND, PARUBND

In [5]:
#mypst = pyemu.pst.pst_utils.pst_from_io_files(
#    tpl_files=[tplfile], in_files=[infile], ins_files=[insfile], out_files=[obsfile])
mypst = pyemu.helpers.pst_from_io_files(
    tpl_files=[tplfile,tplfile2], in_files=[infile,infile2], ins_files=[insfile1,insfile2], out_files=[obsfile1,obsfile2])

In [6]:
print (mypst.observation_data.head())
print (mypst.observation_data.info())

                                obsnme    obsval  weight  obgnme
aetc13-301_2013_04  aetc13-301_2013_04  3.531105     1.0  obgnme
aetc13-301_2013_05  aetc13-301_2013_05  4.857858     1.0  obgnme
aetc13-301_2013_06  aetc13-301_2013_06  5.542544     1.0  obgnme
aetc13-301_2013_07  aetc13-301_2013_07  5.516839     1.0  obgnme
aetc13-301_2013_08  aetc13-301_2013_08  5.081783     1.0  obgnme
<class 'pandas.core.frame.DataFrame'>
Index: 14006 entries, aetc13-301_2013_04 to ro_9_7027720_yr2018
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   obsnme  14006 non-null  object 
 1   obsval  14006 non-null  float64
 2   weight  14006 non-null  float64
 3   obgnme  14006 non-null  object 
dtypes: float64(2), object(2)
memory usage: 1.2+ MB
None


In [7]:
print (mypst.parameter_data.tail())
print (mypst.parameter_data.info())
# For some reason this is not the same number of pars as in the par info notebook.

                    parnme partrans parchglim  parval1       parlbnd  \
wetf_d_tew      wetf_d_tew      log    factor      1.0  1.100000e-10   
wetf_height    wetf_height      log    factor      1.0  1.100000e-10   
wetf_kcb-mid  wetf_kcb-mid      log    factor      1.0  1.100000e-10   
wht_kcb-mid    wht_kcb-mid      log    factor      1.0  1.100000e-10   
wtem_kcb-mid  wtem_kcb-mid      log    factor      1.0  1.100000e-10   

                   parubnd  pargp  scale  offset  dercom  
wetf_d_tew    1.100000e+10  pargp    1.0     0.0       1  
wetf_height   1.100000e+10  pargp    1.0     0.0       1  
wetf_kcb-mid  1.100000e+10  pargp    1.0     0.0       1  
wht_kcb-mid   1.100000e+10  pargp    1.0     0.0       1  
wtem_kcb-mid  1.100000e+10  pargp    1.0     0.0       1  
<class 'pandas.core.frame.DataFrame'>
Index: 494 entries, bare_a_cn to wtem_kcb-mid
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   parnme     

### Add my own OBS and PAR data to the PST object:
### Adding in SW obs info
**Will need another section for bounds on the Irrigation multipliers...**

In [8]:
# Get my own OBS data....
# Uses output from notebook 4.Set and Update OBS weights

allobs = pd.read_csv(os.path.join(OBSDIR,'ALLOBS.INFO.for.pyemu.csv'))

del allobs['Unnamed: 0']
print (allobs.info())
print (allobs.tail(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31738 entries, 0 to 31737
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ObsName    31738 non-null  object 
 1   Value      31666 non-null  float64
 2   OBS_group  31738 non-null  object 
 3   variance   31738 non-null  float64
 4   WEIGHT     31738 non-null  float64
dtypes: float64(3), object(2)
memory usage: 1.2+ MB
None
                   ObsName     Value   OBS_group  variance    WEIGHT
31728  ro_9_7027720_sep-10  0.022857  RO_monthly  0.000024  2.343327
31729  ro_9_7027720_sep-11  0.198571  RO_monthly  0.000214  2.343327
31730  ro_9_7027720_sep-12  0.331429  RO_monthly  0.001048  2.343327
31731  ro_9_7027720_sep-13  0.241429  RO_monthly  0.000414  2.343327
31732  ro_9_7027720_sep-14  0.562857  RO_monthly  0.001490  2.343327
31733  ro_9_7027720_sep-15  0.064286  RO_monthly  0.000062  2.343327
31734  ro_9_7027720_sep-16  0.031429  RO_monthly  0.000081  2.343327
317

In [9]:
# Join my OBS data with the .pst obs data:
# This makes sure everything is sorted correctly...
# fewer values than the .csv file because a lot that were generated are zeros
newpd = pd.merge(mypst.observation_data, allobs, how='left', left_on='obsnme', right_on='ObsName')
print (newpd.info())
# The unmatched values are SW vals where there wasn't any ACTUAL gage data.
# Need to ZERO weight these, since there's nothing to compare them to

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14006 entries, 0 to 14005
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   obsnme     14006 non-null  object 
 1   obsval     14006 non-null  float64
 2   weight     14006 non-null  float64
 3   obgnme     14006 non-null  object 
 4   ObsName    13075 non-null  object 
 5   Value      13075 non-null  float64
 6   OBS_group  13075 non-null  object 
 7   variance   13075 non-null  float64
 8   WEIGHT     13075 non-null  float64
dtypes: float64(5), object(4)
memory usage: 1.1+ MB
None


In [10]:
## Unweight non-matched values:
newpd.loc[newpd.ObsName.isnull(), 'WEIGHT'] = 0
newpd.loc[newpd.ObsName.isnull(), 'OBS_group'] = 'NO_SW_Data'
newpd.loc[newpd.ObsName.isnull(), 'Value'] = -999

#newpd.loc[newpd.ObsName.isnull()].to_csv('MISSING_SW_OBS.csv')

In [11]:
# with the sort correct, this works:
mypst.observation_data['obsval'] = newpd.loc[:,'Value'].values
mypst.observation_data['weight'] = newpd.loc[:,'WEIGHT'].values
mypst.observation_data['obgnme'] = newpd.loc[:,'OBS_group'].values   # Can assign other groups to test as needed....

print (mypst.observation_data.head(10))

                                obsnme  obsval    weight    obgnme
aetc13-301_2013_04  aetc13-301_2013_04    4.02  4.285403  aet-corn
aetc13-301_2013_05  aetc13-301_2013_05    4.55  3.377105  aet-corn
aetc13-301_2013_06  aetc13-301_2013_06    4.99  3.143509  aet-corn
aetc13-301_2013_07  aetc13-301_2013_07    6.04  6.887002  aet-corn
aetc13-301_2013_08  aetc13-301_2013_08    5.82  3.371081  aet-corn
aetc13-301_2013_09  aetc13-301_2013_09    1.99  6.801503  aet-corn
aetc13-302_2013_04  aetc13-302_2013_04    3.58  4.285403  aet-corn
aetc13-302_2013_05  aetc13-302_2013_05    4.22  3.377105  aet-corn
aetc13-302_2013_06  aetc13-302_2013_06    5.87  3.143509  aet-corn
aetc13-302_2013_07  aetc13-302_2013_07    6.13  6.887002  aet-corn


### PARAMETERS.... from LU and IRR tables

In [12]:
# Get my own lookup table PAR data....
# Made this new 8/10/2020

pardata1 = pd.read_csv(os.path.join(PARDIR,'MAP_parlist_MOREPARS.csv')).drop_duplicates(['ParName','ParGroup'])
# Recast to lower case:
pardata1['ParName'] = pardata1['ParName'].str.lower()
print (pardata1.info())
print (pardata1.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 493
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  494 non-null    int64  
 1   ParName     494 non-null    object 
 2   ParGroup    494 non-null    object 
 3   ParVal      494 non-null    float64
 4   ParUBND     494 non-null    float64
 5   ParLBND     494 non-null    float64
 6   Adjust      494 non-null    object 
dtypes: float64(3), int64(1), object(3)
memory usage: 30.9+ KB
None
   Unnamed: 0     ParName ParGroup   ParVal  ParUBND  ParLBND Adjust
0           0   bare_a_cn       cn  81.4000  90.2000  55.7000    YES
1           1   bare_b_cn       cn  88.4308  93.7154  59.2154    YES
2           2   bare_c_cn       cn  92.3554  95.6777  61.1777    YES
3           3   bare_d_cn       cn  93.8992  96.4496  61.9496    YES
4           4  bare_a_mni      mni   3.7500   7.5000   0.9375    YES


In [13]:
print (mypst.parameter_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 494 entries, bare_a_cn to wtem_kcb-mid
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   parnme     494 non-null    object 
 1   partrans   494 non-null    object 
 2   parchglim  494 non-null    object 
 3   parval1    494 non-null    float64
 4   parlbnd    494 non-null    float64
 5   parubnd    494 non-null    float64
 6   pargp      494 non-null    object 
 7   scale      494 non-null    float64
 8   offset     494 non-null    float64
 9   dercom     494 non-null    int32  
dtypes: float64(5), int32(1), object(4)
memory usage: 40.5+ KB
None


In [14]:
# Join these together:
pardf = pd.merge(mypst.parameter_data, pardata1, how='left', left_on='parnme', right_on='ParName')
#print (pardf.info())

# Checking for mis-matched values between the merged datasets:
# THIS ALSO LISTS THE NEW PARAMETERS FROM THE IRRIGATION MULTIPLIERS....

print (pardf.loc[pardf.parnme.isnull()])
# Assign values below....

Empty DataFrame
Columns: [parnme, partrans, parchglim, parval1, parlbnd, parubnd, pargp, scale, offset, dercom, Unnamed: 0, ParName, ParGroup, ParVal, ParUBND, ParLBND, Adjust]
Index: []


In [15]:
# JOIN the PARDATA into the pst object:
# Join to pardf....

# with the sort correct, this works:
mypst.parameter_data['parval1'] = pardf.loc[:,'ParVal'].values
mypst.parameter_data['parlbnd'] = pardf.loc[:,'ParLBND'].values
mypst.parameter_data['parubnd'] = pardf.loc[:,'ParUBND'].values
mypst.parameter_data['pargp'] = pardf.loc[:,'ParGroup'].values   


print(mypst.parameter_data.head())

                parnme partrans parchglim    parval1    parlbnd    parubnd  \
bare_a_cn    bare_a_cn      log    factor  81.400000  55.700000  90.200000   
bare_a_mni  bare_a_mni      log    factor   3.750000   0.937500   7.500000   
bare_a_rew  bare_a_rew      log    factor   0.196000   0.019600   0.294000   
bare_a_rz    bare_a_rz      log    factor   2.440315   1.220158   3.660473   
bare_a_tew  bare_a_tew      log    factor   0.354000   0.035400   0.531000   

           pargp  scale  offset  dercom  
bare_a_cn     cn    1.0     0.0       1  
bare_a_mni   mni    1.0     0.0       1  
bare_a_rew   rew    1.0     0.0       1  
bare_a_rz     rz    1.0     0.0       1  
bare_a_tew   tew    1.0     0.0       1  


In [16]:
print (mypst.parameter_data.loc[mypst.parameter_data.parval1.isnull()])
# Fix these later if there are any...

Empty DataFrame
Columns: [parnme, partrans, parchglim, parval1, parlbnd, parubnd, pargp, scale, offset, dercom]
Index: []


## Setting parameter transformation info (setting non-adjustable parameters, etc.)


In [17]:
# mess with pardf first, then copy into mypst.parameter_data
# Have everything that can vary be LOG transformed (this is what is recommended)
pardf['PARTRANS'] = 'x'
pardf.loc[pardf.Adjust == 'n', 'PARTRANS'] = 'fixed'
pardf.loc[pardf.Adjust == 'YES', 'PARTRANS'] = 'log'

In [18]:
mypst.parameter_data['partrans'] = pardf.loc[:,'PARTRANS'].values
# How many are adjustable?
print (pardf.loc[pardf['Adjust']=='YES'].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 494 entries, 0 to 493
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   parnme      494 non-null    object 
 1   partrans    494 non-null    object 
 2   parchglim   494 non-null    object 
 3   parval1     494 non-null    float64
 4   parlbnd     494 non-null    float64
 5   parubnd     494 non-null    float64
 6   pargp       494 non-null    object 
 7   scale       494 non-null    float64
 8   offset      494 non-null    float64
 9   dercom      494 non-null    int32  
 10  Unnamed: 0  494 non-null    int64  
 11  ParName     494 non-null    object 
 12  ParGroup    494 non-null    object 
 13  ParVal      494 non-null    float64
 14  ParUBND     494 non-null    float64
 15  ParLBND     494 non-null    float64
 16  Adjust      494 non-null    object 
 17  PARTRANS    494 non-null    object 
dtypes: float64(8), int32(1), int64(1), object(8)
memory usage: 71.4+ KB
No

Temporary testing of changing parameter values....

In [19]:
mypst.parameter_data.pargp.unique()

array(['cn', 'mni', 'rew', 'rz', 'tew', 'kcb-mid', 'height'], dtype=object)

### Some cleanup after running PESTCHEK:

### More cleanup after pestchek.exe

### Checking for pars set at or beyond bounds, fix if needed:

In [20]:
## CHECK FOR PARAMETERS ABOVE UPPER BOUND....  If not zero, need to fix.

mypst.parameter_data.loc[((mypst.parameter_data.parval1 > mypst.parameter_data.parubnd))] 

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom


In [21]:
## CHECK FOR PARAMETERS **AT** UPPER BOUND.... (and adjustable)
mypst.parameter_data.loc[((mypst.parameter_data.parval1 == mypst.parameter_data.parubnd) & (mypst.parameter_data.partrans == 'log'))] 

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom


In [22]:
## CHECK for parameters equal to their lower bounds and adjustable...
# This will cause problems for the regularization.
# If not zero, need to fix:

mypst.parameter_data.loc[((mypst.parameter_data.parval1 == mypst.parameter_data.parlbnd) & (mypst.parameter_data.partrans == 'log'))] 


,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom


In [23]:
# Need to check for nans in dataframe.... If there are any, need to fix:
mypst.parameter_data.isnull().any()

parnme       False
partrans     False
parchglim    False
parval1      False
parlbnd      False
parubnd      False
pargp        False
scale        False
offset       False
dercom       False
dtype: bool

In [24]:
mypst.parameter_data.loc[mypst.parameter_data.parval1.isnull()] 

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom


## Copy FINAL versions of par files to PEST directory:

In [25]:
# These will be copied over to YETI for running
# E:\UMID_Data\0188_offline\modeldev\PEST\PEST_FILE_UPDATES
import shutil

In [26]:
insfiles = [f for f in os.listdir(OBSDIR) if f.endswith('.ins')]
print (insfiles)
for f in insfiles:
    shutil.copy(os.path.join(OBSDIR, f), outdir)

['MAP_simulated_et_obs.ins', 'MAP_simulated_et_obs_ALL_ET.ins', 'MAP_simulated_et_obs_ALL_ET_sorted.ins', 'MAP_simulated_obs_SW.ins']


## Final setting of commands and files, NOPTMAX:

In [27]:
mypst.control_data.noptmax = -1
mypst.control_data.formatted_values

name
rstfle                        restart
pestmode                   estimation
npar                                0
nobs                                0
npargp                              0
nprior                              0
nobsgp                              0
maxcompdim                          0
ntplfle                             0
ninsfle                             0
precis                         single
dpoint                          point
numcom                              1
jacfile                             0
messfile                            0
obsreref                   noobsreref
rlambda1                 2.000000E+01
rlamfac                 -3.000000E+00
phiratsuf                3.000000E-01
phiredlam                1.000000E-02
numlam                             -7
jacupdate                         999
lamforgive                 lamforgive
derforgive               noderforgive
relparmax                1.000000E+01
facparmax                1.000000E+01
facorig

In [28]:
mypst.model_command = './swb_forward_run_ALL_obs.sh'      # This runs swb and everything else
mypst.control_data.noptmax = 0

### Make sure these are updated:

In [29]:
#(landuse_version + '.txt')
#(landuse_version + '.tpl')

In [30]:
mypst.template_files = [landuse_version + '.tpl', irrigation_version + '.tpl']
mypst.input_files = [landuse_version + '.txt', irrigation_version + '.txt']

mypst.instruction_files = ['MAP_simulated_et_obs_ALL_ET_sorted.ins','MAP_simulated_obs_SW.ins']
mypst.output_files = ['swbstats_simulated_obs_et_ALL_sorted.obs','swbstats_simulated_obs_SW.obs']

In [31]:
mypst.model_command
#mypst.model_input/output

['./swb_forward_run_ALL_obs.sh']

### Write output:

In [32]:
mypst.write(os.path.join(outdir,'MAP_allobs_calib_run_allpars.pst'))
# Need to edit line 8 of this to match prior versions that worked - remove last two variables.

noptmax:0, npar_adj:494, nnz_obs:13075


In [33]:
mypst.nobs

14006